In [2]:
import pandas as pd
import numpy as np
import copy

In [3]:
master = pd.read_pickle("enron_random_clean1_senders.pkl")

In [4]:
# master = master[master.nSigBlocks > 0]
# master = master[master.filename != "97.txt"]

In [5]:
len(master)

2989

# Convert to sklear-crfsuite format
each sentence should be converted to a list of dicts

In [6]:
import sklearn_crfsuite

In [7]:
def line2features(index, row, source):
    features = row_to_dict(row)

    # There is a previous one
    if row.lineNo > 1:
        try:
            prev_line = source.iloc[index - 1,]
        except IndexError as ex:
            print(index)
            print(row)
        features.update(row_to_dict(prev_line, "-1:"))
    else:
        features['BOS'] = True

    if (row.posFromEnd > 0) and (index < len(source.index) - 1):
        next_line = source.iloc[index + 1,]
        features.update(row_to_dict(next_line, "+1:"))
    else:
        features['EOS'] = True
    return features

In [8]:
def line2features_shift(df):
    df = df.loc[:, featured_columns]
    for col in featured_columns:
        df[f"-1:{col}"] = df[col].shift(1)
        df[f"+1:{col}"] = df[col].shift(-1)
    df["json"] = df.apply(lambda x: x.dropna().to_json(), axis=1)
    return df["json"].tolist()

In [9]:
def get_features(df, files):
    result = []
    for f in files:        
        curr = df[df.filename == f]
        result.append([line2features(index, row, curr) for index, (idx, row) in enumerate(curr.iterrows())])
    return result

In [10]:
def row_to_dict(row, prefix=""):
    features = {}
    for col in featured_columns:
        features[f"{prefix}{col}"] = row[col]
    return features

In [11]:
def get_labels(df, files):
    result = []
    for f in files:        
        curr = df[df.filename == f]
        result.append([row.entity for index, (idx, row) in enumerate(curr.iterrows())])
    return result

In [12]:
def to_crf_suite(df):
    print(df.filename)
    chunks = []
    labels = []
    features = []
    for name, group in df.groupby("filename"):
        print(name)
        labels.append([row.entity for index, (idx, row) in enumerate(group.iterrows())])
        features.append([line2features(index, row, group) for index, (idx, row) in enumerate(group.iterrows())])        
        chunks.append(group)   
    new_df = pd.concat(chunks, ignore_index=True)
    return new_df, features, labels

In [13]:
def to_crf_suite_shifting(df):
    chunks = []
    labels = []
    features = []
    for name, group in df.groupby("filename"):
        labels.append(group.entity)        
        features.append(line2features_shift(group))
        chunks.append(group)   
    new_df = pd.concat(chunks, ignore_index=True)
    return new_df, features, labels

In [14]:
excluded_columns = ["line", 
                    "filename", 
                    "entity", 
                    "label", 
                    "nSig", 
                    "firstchar", 
                    'nlines',
                     'len_avg',
                    'len_min',
                    'len_max',
                    'nBlanks',
                    'nNonBlanks',
                    'nSigBlocks',
                    'pred_label',
                    "next_label",
                    "sigToLinesRatio",
                    "pred_file",
                    "next_file",
                    "lineNo",
                    "len"
                    
 ]
featured_columns = [c for c in master.columns if c not in excluded_columns]

In [15]:
filenames = master.filename.unique()

In [16]:
from sklearn.model_selection import train_test_split
train_filenames, test_filenames = train_test_split(filenames, test_size=0.2, random_state=42)

In [17]:
train = master[master.filename.isin(train_filenames)]
test = master[master.filename.isin(test_filenames)]

In [18]:
train = train.sort_values(by=["filename", "lineNo"])
test = test.sort_values(by=["filename", "lineNo"])

In [19]:
# all should be empty if all match
print([(x, y) for x, y in zip(sorted(train_filenames), sorted(train.filename.unique())) if x != y])
print([(x, y) for x, y in zip(sorted(test_filenames), sorted(test.filename.unique())) if x != y])
print([x for x in train_filenames if x in test_filenames])

[]
[]
[]


In [20]:
train, X_train, y_train = to_crf_suite(train)
test, X_test, y_test = to_crf_suite(test)

0       allen-p_all_documents_113.txt
1       allen-p_all_documents_113.txt
2       allen-p_all_documents_113.txt
3       allen-p_all_documents_113.txt
4       allen-p_all_documents_113.txt
                    ...              
2984     gay-r_all_documents_46_a.txt
2985     gay-r_all_documents_46_a.txt
2986     gay-r_all_documents_46_a.txt
2987     gay-r_all_documents_46_a.txt
2988     gay-r_all_documents_46_a.txt
Name: filename, Length: 2238, dtype: object
allen-p_all_documents_113.txt
allen-p_all_documents_171.txt
allen-p_all_documents_496.txt
allen-p_all_documents_505.txt
allen-p_all_documents_575.txt
allen-p_all_documents_595.txt
allen-p_all_documents_599.txt
allen-p_all_documents_90.txt
arnold-j_all_documents_1027.txt
arnold-j_all_documents_454_a.txt
arnold-j_all_documents_687.txt
arora-h_all_documents_58.txt
arora-h_all_documents_64.txt
badeer-r_all_documents_109.txt
badeer-r_all_documents_119.txt
badeer-r_all_documents_128.txt
badeer-r_all_documents_257.txt
badeer-r_all_document

In [21]:
len(X_test)

31

In [22]:
len(test_filenames)

31

In [23]:
from collections import Counter
Counter(train.entity)

Counter({'no_entity': 1894, 'signature': 344})

In [24]:
crf = sklearn_crfsuite.CRF(c1=0.17)
crf.fit(X_train, y_train);

In [25]:
crf.classes_

['no_entity', 'signature']

In [26]:
from sklearn_crfsuite import metrics
y_pred = crf.predict(X_test)
metrics.flat_f1_score(y_test, y_pred, average='binary', pos_label='signature')

0.8907563025210085

In [230]:
Counter(flatten(y_pred))

Counter({'no_entity': 352, 'signature': 141})

In [231]:
Counter(flatten(y_test))

Counter({'no_entity': 356, 'signature': 137})

In [232]:
from sklearn_crfsuite.utils import flatten

In [233]:
print(metrics.flat_classification_report(y_test, y_pred, labels=list(crf.classes_)))

              precision    recall  f1-score   support

   no_entity       0.98      0.97      0.97       356
   signature       0.92      0.95      0.94       137

    accuracy                           0.96       493
   macro avg       0.95      0.96      0.95       493
weighted avg       0.96      0.96      0.96       493



G:\Programs\Anaconda2\envs\thesis-enron\lib\site-packages\sklearn\utils\validation.py:70: FutureWarning: Pass labels=['no_entity', 'signature'] as keyword args. From version 0.25 passing these as positional arguments will result in an error
  FutureWarning)


# Improve with blocking

In [275]:
y_pred = crf.predict(X_test)

In [290]:
y_pred_blocks = copy.deepcopy(y_pred)
from nltk import ngrams
for doc in y_pred_blocks:
    for i, x in enumerate(doc):
        try:
            prev = doc[i-1]
            next_ = doc[i+1]
            next_next_ = doc[i+2]
        except IndexError as ex:
            prev = None
            next_ = None

        if prev and next_ and prev == "signature" and next_ == "signature" and next_next_ == "signature" and x == "no_entity":
            doc[i] = "signature"
            print([doc[i-1], doc[i], doc[i+1], doc[i+2]])

['signature', 'signature', 'signature', 'signature']


In [284]:
[(v, w, t) for x, y, truth in zip(y_pred, y_pred_blocks, y_test) for v, w, t in zip(x, y, truth) if v != w]

[('no_entity', 'signature', 'signature'),
 ('no_entity', 'signature', 'no_entity'),
 ('no_entity', 'signature', 'no_entity'),
 ('no_entity', 'signature', 'signature'),
 ('no_entity', 'signature', 'no_entity'),
 ('no_entity', 'signature', 'no_entity')]

In [287]:
metrics.flat_f1_score(y_test, y_pred, average='binary', pos_label='signature')

0.935251798561151

In [291]:
metrics.flat_f1_score(y_test, y_pred_blocks, average='binary', pos_label='signature')

0.9318996415770608

In [296]:
test["pred"] = flatten(y_pred)

In [297]:
test.to_pickle("crf_test_results.pkl")

In [141]:
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GroupKFold

In [144]:
# use the same metric for evaluation
from sklearn.metrics import make_scorer
f1_scorer = make_scorer(metrics.flat_f1_score, average='binary', pos_label='signature')

In [148]:
scores = cross_val_score(sklearn_crfsuite.CRF(), 
                X_test, y_test,
                cv=10,
                scoring=f1_scorer,
                n_jobs=-1, 
                verbose=10 )

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.
G:\Programs\Anaconda2\envs\thesis-enron\lib\site-packages\sklearn\base.py:213: FutureWarning: From version 0.24, get_params will raise an AttributeError if a parameter cannot be retrieved as an instance attribute. Previously it would return None.
  FutureWarning)
[Parallel(n_jobs=-1)]: Done   3 out of  10 | elapsed:    2.9s remaining:    6.9s
[Parallel(n_jobs=-1)]: Done   5 out of  10 | elapsed:    4.6s remaining:    4.6s
[Parallel(n_jobs=-1)]: Done   7 out of  10 | elapsed:    6.4s remaining:    2.7s
[Parallel(n_jobs=-1)]: Done  10 out of  10 | elapsed:    8.2s finished


In [149]:
scores.mean()

0.9098372795131461

In [26]:
import eli5
eli5.explain_weights(crf, top=50)

G:\Programs\Anaconda2\envs\thesis-enron\lib\site-packages\sklearn\utils\deprecation.py:143: FutureWarning: The sklearn.metrics.scorer module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.metrics. Anything that cannot be imported from sklearn.metrics is now part of the private API.
  warnings.warn(message, FutureWarning)
G:\Programs\Anaconda2\envs\thesis-enron\lib\site-packages\sklearn\utils\deprecation.py:143: FutureWarning: The sklearn.feature_selection.base module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.feature_selection. Anything that cannot be imported from sklearn.feature_selection is now part of the private API.
  warnings.warn(message, FutureWarning)


NameError: name 'crf' is not defined

# Grid Search

In [27]:
import scipy
from sklearn.metrics import make_scorer
from sklearn.model_selection import RandomizedSearchCV
from sklearn_crfsuite import metrics

In [28]:
# define fixed parameters and parameters to search
crf = sklearn_crfsuite.CRF(
    algorithm='lbfgs'
)
params_space = {
    'c1': scipy.stats.expon(scale=0.5),
    'c2': scipy.stats.expon(scale=0.05)
}

# use the same metric for evaluation
f1_scorer = make_scorer(metrics.flat_f1_score, average='binary', pos_label='signature')

# search
rs = RandomizedSearchCV(crf, params_space,
                        cv=3,
                        verbose=1,
                        n_jobs=-1,
                        n_iter=50,
                        scoring=f1_scorer,
                        refit=False)
rs.fit(X_train, y_train)

G:\Programs\Anaconda2\envs\thesis-enron\lib\site-packages\sklearn\base.py:213: FutureWarning: From version 0.24, get_params will raise an AttributeError if a parameter cannot be retrieved as an instance attribute. Previously it would return None.
  FutureWarning)
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.


Fitting 3 folds for each of 50 candidates, totalling 150 fits


[Parallel(n_jobs=-1)]: Done  26 tasks      | elapsed:  1.9min
[Parallel(n_jobs=-1)]: Done 150 out of 150 | elapsed: 12.0min finished


RandomizedSearchCV(cv=3, estimator=CRF(algorithm='lbfgs', keep_tempfiles=None),
                   n_iter=50, n_jobs=-1,
                   param_distributions={'c1': <scipy.stats._distn_infrastructure.rv_frozen object at 0x0000026B2E524488>,
                                        'c2': <scipy.stats._distn_infrastructure.rv_frozen object at 0x0000026B2E9CBF88>},
                   refit=False,
                   scoring=make_scorer(flat_f1_score, average=binary, pos_label=signature),
                   verbose=1)

In [33]:
best = rs.best_estimator_

AttributeError: 'RandomizedSearchCV' object has no attribute 'best_estimator_'

In [32]:
best = rs.fit(X_train, y_train)

Fitting 3 folds for each of 50 candidates, totalling 150 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done  26 tasks      | elapsed:  2.3min
[Parallel(n_jobs=-1)]: Done 150 out of 150 | elapsed: 12.9min finished


In [214]:
rs

RandomizedSearchCV(cv=3, estimator=CRF(algorithm='lbfgs', keep_tempfiles=None),
                   n_iter=50, n_jobs=-1,
                   param_distributions={'c1': <scipy.stats._distn_infrastructure.rv_frozen object at 0x0000023C15E2CF88>,
                                        'c2': <scipy.stats._distn_infrastructure.rv_frozen object at 0x0000023C15E2C1C8>},
                   refit=False,
                   scoring=make_scorer(flat_f1_score, average=binary, pos_label=signature),
                   verbose=1)

In [30]:
rs.best_score_

0.8364048389260947

In [31]:
print('best params:', rs.best_params_)
print('best CV score:', rs.best_score_)
print('model size: {:0.2f}M'.format(rs.best_estimator_.size_ / 1000000))

best params: {'c1': 0.3133135499952666, 'c2': 0.006111656622502702}
best CV score: 0.8364048389260947


AttributeError: 'RandomizedSearchCV' object has no attribute 'best_estimator_'